In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import numpy as np

In [2]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=2,
                          num_workers=0,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=4,
                          num_workers=0,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=1,
                          num_workers=0,
                          shuffle=True)

In [3]:
def iou(pred, target):
#     ious = []
    cls = [7,8,11,12,13,17,19,20,21,22,23,24,25,26,27,28,31,32,33]
    
    pred = pred[:,cls].cpu().numpy()
    target = target[:,cls].cpu().numpy()
#     print(pred.shape, target.shape)
    
    intersection = np.sum(pred * target, axis=(0,2,3))
    union = np.sum(pred ^ target, axis=(0,2,3)) + intersection
    
    iou = intersection / union
    
#     for cls in range(n_class):
#         if cls not in [0,1,2,3,4,5,6,9,10,14,15,16,18,29,30]:
#             intersection =(pred[:,cls,:,:] & target[:,cls,:,:]).float().sum()
#             union = (pred[:,cls,:,:] | target[:,cls,:,:]).float().sum()  
#             if union == 0:
#                 ious.append(float('nan'))  # if there is no ground truth, do not include in evaluation
#             else:
#                 ious.append(float(intersection/union))
#     del intersection
#     del union
#     torch.cuda.empty_cache()
    
    return iou

In [4]:
def pixel_acc(pred, target):
#     print(pred.shape)
#     cls = [0,1,2,3,4,5,6,9,10,14,15,16,18,29,30]
    cls = [7,8,11,12,13,17,19,20,21,22,23,24,25,26,27,28,31,32,33]
    pred = pred[:,cls,:,:]
    target = target[:,cls,:,:]
    
    matches = (pred == target)
    pixelacc = matches.sum().item()/matches.nelement()
#             matching_pixels =  !(pred.int() ^ target.int())
#     matching_pixels = np.array(matching_

    del pred, target
    torch.cuda.empty_cache()
    return pixelacc

In [5]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.constant(m.bias, 0)
        
epochs     = 100
criterion = nn.CrossEntropyLoss()
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  after removing the cwd from sys.path.


In [6]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for idx, (X, tar, Y) in enumerate(train_loader):
            if idx == 50:
                break
#             print(epoch,idx)
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
            else:
                inputs, labels = X,Y

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if idx % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, idx, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    pixelAcc = 0
    count = 0
    ious = np.zeros((1,19))
    loss = 0
    with torch.no_grad():
        for idx, (X, tar, Y) in enumerate(val_loader):
            if idx == 50:
                break
            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
                targets = tar.cuda()
            else:
                inputs, labels, targets = X,Y, tar
            outputs = fcn_model(inputs)
#         outputs = outputs.cpu()
            outputs = F.softmax(outputs,dim=0)
            max_idx = torch.argmax(outputs, 0, keepdim=True)
            one_hot = torch.FloatTensor(outputs.shape)
            one_hot.zero_()
            one_hot = one_hot.cuda()
            one_hot.scatter_(0, max_idx, 1)
#             del max_idx
            
            ious += iou(one_hot.int(),targets.int())
#             loss+=criterion(outputs, labels).item()
#         print(ious)
#             pixelAcc += pixel_acc(one_hot, targets)
            count+=1
            print(idx)
#         print(pixelAcc)
    ious = ious / idx
    loss /= idx
    pixelAcc /= idx
    print("Validation loss {}, pixelacc {}".format(loss, ious))
#     [iou/count for iou in ious]
    return loss, pixelAcc, ious
        
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    pass
    
if __name__ == "__main__":
#     val(0)  # show the accuracy before training
    train()

epoch0, iter0, loss: 3.938131809234619
epoch0, iter10, loss: 3.0921471118927
epoch0, iter20, loss: 2.3963098526000977
epoch0, iter30, loss: 2.696580410003662
epoch0, iter40, loss: 2.6036646366119385
Finish epoch 0, time elapsed 39.459245920181274


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-pa

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Validation loss 0.0, pixelacc [[0.17859105 0.05537089 0.10224867 0.00510893 0.00772294 0.01016642
  0.0024755  0.00624833 0.13375172 0.00561955 0.0325178  0.01124734
  0.00207064 0.02569109 0.00350772 0.00458949 0.00317577 0.00147982
  0.00659147]]
epoch1, iter0, loss: 2.3643624782562256
epoch1, iter10, loss: 2.259901523590088
epoch1, iter20, loss: 2.2120070457458496
epoch1, iter30, loss: 2.031625986099243
epoch1, iter40, loss: 1.8568235635757446
Finish epoch 1, time elapsed 42.38836860656738
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Validation loss 0.0, pixelacc [[0.18436066 0.04695305 0.11170277 0.00895301 0.00799186 0.01001851
  0.00192562 0.00791599 0.08179536 0.00749006 0.058089   0.01075246
  0.00181621 0.06824374 0.00190659 0.00555658 0.00021552

KeyboardInterrupt: 